In [1]:
import pandas as pd
import os

In [2]:
s3_prefix = "s3://aegovan-data/pubmed_asbtract/predictions_multi_80/"
s3_data ="s3://aegovan-data/human_output/human_interactions_ppi_v2.json"

In [3]:
local_temp = "temp"
local_temp_pred_dir = os.path.join( local_temp, "pred_results")
local_temp_wk_dir = os.path.join( local_temp, "wk")

In [4]:
!rm -rf $local_temp
!mkdir -p $local_temp_pred_dir
!mkdir -p $local_temp_wk_dir

In [5]:
#!aws s3 cp s3://aegovan-data/pubmed_asbtract/predictions_multi_95/pubmed19n0538.json.txt.json.prediction.json .

In [6]:
import boto3
import glob
from multiprocessing.dummy import Pool as ThreadPool
import argparse
import datetime 
import os


def uploadfile(localpath, s3path):
        """
Uploads a file to s3
        :param localpath: The local path
        :param s3path: The s3 path in format s3://mybucket/mydir/mysample.txt
        """

        bucket, key = get_bucketname_key(s3path)

        if key.endswith("/"):
            key = "{}{}".format(key, os.path.basename(localpath))
        
        s3 = boto3.client('s3')
        
        s3.upload_file(localpath, bucket, key)

def get_bucketname_key(uripath):
    assert uripath.startswith("s3://")

    path_without_scheme = uripath[5:]
    bucket_end_index = path_without_scheme.find("/")

    bucket_name = path_without_scheme
    key = "/"
    if bucket_end_index > -1:
        bucket_name = path_without_scheme[0:bucket_end_index]
        key = path_without_scheme[bucket_end_index + 1:]

    return bucket_name, key


def download_file(s3path, local_dir):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')
    
    local_file = os.path.join(local_dir, s3path.split("/")[-1])
    

    s3.download_file(bucket, key, local_file)
    
def download_object(s3path):
    bucket, key = get_bucketname_key(s3path)
    
    s3 = boto3.client('s3')    

    s3_response_object = s3.get_object(Bucket=bucket, Key=key)
    object_content = s3_response_object['Body'].read()
    
    return len(object_content)



def list_files(s3path_prefix):
    assert s3path_prefix.startswith("s3://")
    assert s3path_prefix.endswith("/")
    
    bucket, key = get_bucketname_key(s3path_prefix)
    
   
   
    s3 = boto3.resource('s3')
    
    bucket = s3.Bucket(name=bucket)

    return ( (o.bucket_name, o.key) for o in bucket.objects.filter(Prefix=key))





def upload_files(local_dir, s3_prefix, num_threads=20):    
    input_tuples = ( (f,  s3_prefix) for f in glob.glob("{}/*".format(local_dir)))
    
    with ThreadPool(num_threads) as pool:
        pool.starmap(uploadfile, input_tuples)
    


def download_files(s3_prefix, local_dir, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        
        

def download_objects(s3_prefix, num_threads=20):    
    s3_files = ( "s3://{}/{}".format(s3_bucket,s3_key) for s3_bucket, s3_key in list_files(s3_prefix))
    
    with ThreadPool(num_threads) as pool:
        results = pool.map(download_object, s3_files)
        
    return sum(results)/1024
        

def get_directory_size(start_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def get_s3file_size(bucket, key):
    s3 = boto3.client('s3')
    response = s3.head_object(Bucket=bucket, Key=key)
    size = response['ContentLength']
    return size
    
def download_files_min_files(s3_prefix, local_dir, min_file_size=310, num_threads=20):    
    input_tuples = ( ("s3://{}/{}".format(s3_bucket,s3_key),  local_dir) for s3_bucket, s3_key in list_files(s3_prefix) if get_s3file_size(s3_bucket, s3_key) > min_file_size )
    
    with ThreadPool(num_threads) as pool:
        results = pool.starmap(download_file, input_tuples)
        


In [16]:
%%time

download_files(s3_prefix, local_temp_pred_dir)

CPU times: user 29.4 s, sys: 6.38 s, total: 35.7 s
Wall time: 2min 38s


In [17]:
!ls -l $local_temp_dir | wc -l

     150


In [18]:
total = 0
largest_df = None
full_df = None
value_dict={}
for f in os.listdir(local_temp_pred_dir):
    df = pd.read_json(os.path.join(local_temp_pred_dir, f), orient="records", lines=True )
    
    if largest_df is None:
        largest_df = df
    
    if df.shape[0] > largest_df.shape[0]:
        largest_df=df
    
    if full_df is None:
        full_df = df
    else:
        full_df = pd.concat([df, full_df])
        
    prediction_counts = df["predicted"].value_counts().to_dict()
    for k,v in prediction_counts.items():      
        value_dict [k] =  v +  value_dict.get(k, 0)
    
    #total records
    total += df.shape[0]

In [19]:
value_dict

{'phosphorylation': 86137,
 'methylation': 3174,
 'dephosphorylation': 316,
 'acetylation': 52}

In [34]:
full_df.groupby(["predicted"])["predicted_confidence"].max()

predicted
acetylation          0.906370
dephosphorylation    0.962255
methylation          0.946525
phosphorylation      0.991165
Name: predicted_confidence, dtype: float64

In [20]:
total

89679

In [21]:
download_file(s3_data, local_temp_wk_dir)



In [22]:
data_file = os.path.join(local_temp_wk_dir, s3_data.split("/")[-1])
data_training_full_df = pd.read_json(data_file)

In [23]:
data_training_full_df.shape

(3381, 7)

In [24]:
full_df.shape

(89679, 16)

In [25]:
data_training_full_df.head(n=2)

,interactionId,interactionType,isNegative,participants,pubmedId,pubmedTitle,pubmedabstract
0,1585513,phosphorylation,false,"[{'uniprotid': 'Q10728', 'alias': [['mypt1_rat...",17126281,None,Zipper-interacting protein kinase (ZIP kinase)...
1,1585516,phosphorylation,false,"[{'uniprotid': 'O43293-1', 'alias': [['o43293-...",17126281,None,Zipper-interacting protein kinase (ZIP kinase)...


In [26]:
full_df.head(n=2)

,abstract,acetylation,confidence_scores,demethylation,dephosphorylation,deubiquitination,methylation,normalised_abstract,other,participant1Id,participant2Id,phosphorylation,predicted,predicted_confidence,pubmedId,ubiquitination
0,"For the past century, the renin-angiotensin sy...",0.345884,"{'acetylation': -1.0616518736, 'demethylation'...",0.165448,0.678281,0.272083,0.206545,"For the past century, the renin-angiotensin sy...",4.597788,P01019,P23946,86.943618,phosphorylation,0.929718,23176219,0.306509
1,Bone formation is a rarely encountered finding...,0.557860,"{'acetylation': -0.5836479068, 'demethylation'...",0.209011,0.807035,0.421113,0.248991,Bone formation is a rarely encountered finding...,2.604435,P37023,Q9UK05,32.053825,phosphorylation,0.857942,23177617,0.459018


In [27]:
full_df[~full_df.pubmedId.isin(data_training_full_df.pubmedId)].shape

(89210, 16)

In [28]:
full_df["PubmedInTrainingData"] = full_df.pubmedId.isin(data_training_full_df.pubmedId)

In [29]:

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 10000)
pd.set_option('display.max_rows', 100)

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [30]:
full_df.to_csv("predictions.csv", index=False, header=True)

In [31]:
full_df[["abstract", "normalised_abstract", "participant1Id","participant2Id", "pubmedId", "predicted" ,"predicted_confidence" ]].sample(n=20)

,abstract,normalised_abstract,participant1Id,participant2Id,pubmedId,predicted,predicted_confidence
45,"Previous genetic linkage studies identified a locus for susceptibility to prostate cancer called HPCX at Xq27. The candidate region contains two clusters of SPANX genes. The first cluster called SPANX-A/D includes SPANX-A1, SPANX-A2, SPANX-B, SPANX-C, and SPANX-D; the second cluster called SPANX-N includes SPANX-N1, SPANX-N2, SPANX-N3, and SPANX-N4. The SPANX genes encode cancer-testis (CT) specific antigens. Previous studies identified SPANX-B and SPANX-D variants produced by gene conversion events, none of which are associated with X-linked prostate cancer.","Previous genetic linkage studies identified a locus for susceptibility to prostate cancer called 9566 at Xq27. The candidate region contains two clusters of Q9NS26 genes. The first cluster called 30014;64648 includes Q9NS26, Q9NS26, Q9NS25, Q9NY87, and Q9BXN6; the second cluster called Q9NS26-N includes Q5VSR9, Q5MJ10, Q5MJ09, and Q5MJ08. The Q9NS26 genes encode cancer-testis (CT) specific antigens. Previous studies identified Q9NS25 and Q9BXN6 variants produced by gene conversion events, none of which are associated with X-linked prostate cancer.",30014;64648,Q9NS26,17373721,phosphorylation,0.803549
72,"SCA6 is a slowly progressive, late-onset cerebellar ataxia due to a trinucleotide expansion in the CACNA1A gene. We describe two unrelated cases that presented with Parkinsonism and cerebellar ataxia. One case was L-dopa-responsive with a pattern of (18)F-dopa uptake similar to Parkinson's disease, and the second case was not L-dopa-responsive and had an atypical pattern of nigrostriatal dysfunction. We suggest that SCA6, in common with SCA2 and SCA3, may be associated with Parkinsonism attributable to nigral loss and dopaminergic dysfunction. Moreover, isolated cases may be confused with multiple system atrophy.","O00555 is a slowly progressive, late-onset cerebellar ataxia due to a trinucleotide expansion in the O00555 gene. We describe two unrelated cases that presented with Parkinsonism and cerebellar ataxia. One case was L-dopa-responsive with a pattern of (18)F-dopa uptake similar to Parkinson's disease, and the second case was not L-dopa-responsive and had an atypical pattern of nigrostriatal dysfunction. We suggest that O00555, in common with Q99700 and P54252, may be associated with Parkinsonism attributable to nigral loss and dopaminergic dysfunction. Moreover, isolated cases may be confused with multiple system atrophy.",O00555,Q99700,15954136,phosphorylation,0.891431
96,"Induction of drug-metabolizing microsomal cytochromes p450 (p450s) results in a striking proliferation of the smooth endoplasmic reticulum (ER). Overexpression of P450s in yeast and cultured cells produces a similar response. The signals mediating this process are not known but probably involve signal transduction pathways involved in the unfolded protein response (UPR) or the ER overload response (EOR). We have examined the temporal response of specific genes in these pathways and genes globally to overexpression of p450 in cultured cells. Activity of NFkappaB, an EOR component, was substantially increased by overexpression of full-length p450 2C2 or a chimera with the 28-amino acid signal anchor sequence of p450 2C2 in HepG2 cells, and the activation correlated temporally with the accumulation of p450 in the cells. In the UPR pathway, activation of the transcription factor XBP1 by IRE1 also correlated with the accumulation of p450 in the cells, and in contrast, maximum activation of the BiP/grp78 promoter preceded the accumulation. Differential effects of expression of p450 on apoptosis were observed in nonhepatic COS1 and hepatic HepG2 cells. In COS1 cells, apoptosis was induced, and this correlated with sustained activation of the pro-apoptotic JNK pathway, induction of CHOP, and an absence of the increased NFkappaB activity. In HepG2 cells, JNK was only transiently activated, and CHOP